In [10]:
def up_run(fid, data, tc):
    '''
    Universal Portfolio
    '''

    n, m = data.shape #n:rows, m:cols #n:507 m:30
    cum_ret = 1
    cumprod_ret = np.ones((n, 1), float) #507,1

    daily_ret = np.ones((n, 1), float) #507,1

    day_weight = np.ones((m, 1), float)/m  #30,1
    day_weight_o = np.zeros((m, 1), float) # 30,1
    daily_portfolio = np.zeros((n,m)) #507,30

    #print log file head
    print(fid, '-------------------------------------\n')
    print(fid, 'Parameters [tc: %f].\n' % tc)
    print(fid, 'day\t Daily Return\t Total return\n')
    print(1, '-------------------------------------\n')

    '''
          if opts.quiet_mode == False:
            print(1, 'Parameters [tc: %f].\n' % tc)
            print(1, 'day\t Daily Return\t Total return\n')

    '''

        #backtest
    for t in range(n):

            # Calculate t's portfolio at the beginning of t-th trading day
        if t >= 1:
            '''calculate t's portfolio at the beginning of t-th trading day'''
            day_weight = up_kernel(data[0:t-1, :], day_weight_o)

        # Normalize the constraint, always useless
        #print('day_weight is ', day_weight)
        day_weight = np.divide(day_weight, np.sum(day_weight, axis=0)) # elementwise divide and column-sum
        #print('day_weight after is ', day_weight)
        
        daily_portfolio[t,:] = day_weight.conj().T
        #print(daily_portfolio)
            
        # Cal t's return and total return
        daily_ret[t, 0] = np.dot(data[t,:],day_weight)*(1-tc/2*np.sum(np.absolute(day_weight-day_weight_o) ))
        cum_ret = np.dot(cum_ret , daily_ret[t,0])
        cumprod_ret[t, 0] = cum_ret
        
        #Adjust weight[t, :] for the transaction cost issue

        day_weight_o = day_weight * data[t,:].conj().T / daily_ret[t, 0]

        #print(fid, '%d\t%f\t%f\n' % (t, daily_ret[t,0], cumprod_ret[t,0]))
        #print(day_weight)
        #print('------------------------------------------------\n')
        #print(day_weight_o)

        '''
            if opts.quiet_mode == False:
                if np.mod(t, opts.display_interval) == 0:
                      print(1, '%d\t%f\t%f\n' % (t, daily_ret[:t,:1], cumprod_ret[:t,:1]))     
        '''

    
    print(fid, 'Market(tc=%.4f), Cumulative return: %.2f\n'% (tc, cum_ret))
    print(fid, '-------------------------------------\n')
    print(1, 'Market(tc=%.4f), Cumulative return: %.2f\n'% (tc, cum_ret))
    print(1, '-------------------------------------\n')
        
        
def up_kernel(data, weight_o):
    weight = up_expert(data, weight_o)
    return weight
    
def up_expert(data, weight_o):
    del0 = 0.004 # minimum coordinate
    delta = 0.005 # spacing of grid #0.005
    M = 10 #Number of samples
    S = 5 # Number of steps in the random walk
        
    __ , N = data.shape # ph: placeholder, in original matlab code is ~ #N:30
        
    r = np.ones((N,1))/N
    b = np.ones((r.shape))
    #print('shape of b is', b.shape)
        
    allM = np.zeros((N,M)) # take the average of m samples
        
    for m in range(M):
        b = r
        for i in range(S):
            bnew = np.copy(b)
            # choose 1 <= j <= N-1 at random
            j = np.random.randint(low=1, high=N)
            #print('j is ',j)
            #print('N is ', N)
            #print('shape of bnew is', bnew.shape)
            #print('bnew is ', bnew)
            #print('b[N] is ', b[N-1])
                
            # choose X = -1 or 1 randomly
            a = np.random.randint(-1, 1)
            if a == 0:
                a = 1
            #print('a is ', a)
            bnew[j] = b[j]+ (a*delta)
            #print('bnew[j] is ',bnew[j])
            bnew[N-1] = b[N-1] - (a*delta)
            #print('shape of bnew[n-1] is ', bnew[N-1].shape)
            #print('bnew[n-1] is ', bnew[N-1]) #bnew is basically right
            if bnew[j] >= del0 and bnew[N-1]>= del0:
                x = up_expert_findQ(b, data, del0, delta)
                y = up_expert_findQ(bnew, data, del0, delta)
                #print('\nx is ', x) #x,y is almost correct
                #print('\ny is ', y)
                    
                pr = np.minimum(y/x, 1)
                temp = np.random.uniform(0,1)
                if temp < pr:
                    b = bnew
        #print('The shape of b is ', b.shape)
        #print('m is', m)
        #print('The current b is ', b)
        #psudov = b
        #print('psudo vector is', psudov[:,None])
        allM[:,m] = b.conj().T
        #print('The shape of allM is', allM[:,m].shape)
        #print('current allM is ', allM[:,m]) #numeric is right now
            
    weight = np.sum(allM, axis=1)/M # sum of rows axis =1
    #print('weight is', weight) #numeric is right
        
    return weight
    
def up_expert_findQ(b, data, del0, delta):
    __, N = data.shape
        
    P = np.prod(np.dot(data[:,:],b))
    #print('P is ', P) #there is some difference but not very big
    
        
    Q = np.dot(P , np.minimum(1, np.exp((b[N-1]-(2*del0))/((N-1)*delta))))
    #print('The Q is, ', Q)
    return Q
                

        
import numpy as np
import scipy.io as sio
djia = sio.loadmat('djia.mat') # load a test input
print(djia['data'][1:2,:].shape)
up_run(1, djia['data'][1:2,:], 0.5)

(1, 30)
1 -------------------------------------

1 Parameters [tc: 0.500000].

1 day	 Daily Return	 Total return

1 -------------------------------------

[[ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]
 [ 0.03333333]]
------------------------------------------------

[[ 0.04481525  0.04494148  0.04623972  0.04630424  0.04343821  0.04315267
   0.04296737  0.04731441  0.04396997  0.04394047  0.04575704  0.04411007
   0.04401435  0.04459     0.04557092  0.04388821  0.04301915  0.04038059
   0.04395054  0.04414578  0.04443958  0.04605075  0.04523684  0.04382582
   0.04502501  0.0433037   0.04474622  0.0458632

In [17]:
print(5e-3)
print(1e1)
print(0.5e1)

0.005
10.0
5.0


In [1]:
4e-3

0.004

In [3]:
1e1

10.0